## Configuration

In [1]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *
from datetime import date

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

# Old Playlists
playlistsOld = [
    'Summer 2018',
    'Summer 2019',
    'Endless Summer 2020',
    'Endless Summer 2021',
    'Endless Summer 2022',
    'Endless Summer 2023', 
]

# Filepaths
artistsKnownPath = os.path.join('data', 'artistsKnown.csv')
artistsHistoricalPath = os.path.join('data', 'artistsHistorical.csv')

## Log In

In [2]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Read in Known Artists

In [3]:
# Read in known artists from csv
if os.path.exists(artistsKnownPath):
    artistsKnown = pd.read_csv(artistsKnownPath)
else:
    artistsKnown = pd.DataFrame(columns=['artistID', 'following'])

artistsKnown.head()

,artistID,artistName,artistPopularity,artistFollowers,artistGenre,date
0,1URnnhqYAYcrqrcwql10ft,21 Savage,91,16470178,"atl hip hop, hip hop, rap",2024-01-15
1,2RTUTCvo6onsAnheUk3aL9,3 Doors Down,69,4257459,"alternative metal, nu metal, post-grunge, rock",2024-01-15
2,41Q0HrwWBtuUkJc7C1Rp6K,311,60,1002040,"alternative metal, alternative rock, funk meta...",2024-01-15
3,4NiJW4q9ichVqL1aUsgGAN,A Day To Remember,68,2281593,"alternative metal, metalcore, neon pop punk, p...",2024-01-15
4,09hVIj6vWgoCDtT03h8ZCa,A Tribe Called Quest,66,2029407,"conscious hip hop, east coast hip hop, golden ...",2024-01-15


## Read in old playlists

In [4]:
# Empty df
artistsOld = pd.DataFrame()
# Read in all old summer playlists
for playlistName in playlistsOld:
    # Get playlistID
    id_pl = getPlaylistID(credentials, playlistName)
    # Get playlistInfo
    tracks_pl = fetchTracksFromPlaylist(credentials, id_pl)
    # Append to df
    artistsOld = pd.concat([artistsOld, tracks_pl], axis=0)
# Deduplicate
artistsOld = artistsOld.drop_duplicates('artistID', ignore_index=True)[['artistID']]

artistsOld.head()

,artistID
0,2ZRQcIgzPCVaT9XKhXZIzh
1,3ApUX1o6oSz321MMECyIYd
2,44eNlYXYkIGob3NU82tgLq
3,3mJ9GlkLzj8Ka7Z7EQaCMi
4,2cCUtGK9sDU2EoElnk0GNB


## Merge and deduplicate known artists and any from new playlists

In [5]:
artistsKnown = pd.concat([artistsKnown, artistsOld], axis=0).drop_duplicates('artistID', ignore_index=True)
# Mark Liked and Unliked artists
artistsKnown.loc[artistsKnown['artistID'].isin(artistsOld['artistID']), 'following'] = True
artistsKnown.loc[~artistsKnown['artistID'].isin(artistsOld['artistID']), 'following'] = False
artistsKnown

,artistID,artistName,artistPopularity,artistFollowers,artistGenre,date,following
0,1URnnhqYAYcrqrcwql10ft,21 Savage,91.0,16470178.0,"atl hip hop, hip hop, rap",2024-01-15,True
1,2RTUTCvo6onsAnheUk3aL9,3 Doors Down,69.0,4257459.0,"alternative metal, nu metal, post-grunge, rock",2024-01-15,False
2,41Q0HrwWBtuUkJc7C1Rp6K,311,60.0,1002040.0,"alternative metal, alternative rock, funk meta...",2024-01-15,False
3,4NiJW4q9ichVqL1aUsgGAN,A Day To Remember,68.0,2281593.0,"alternative metal, metalcore, neon pop punk, p...",2024-01-15,False
4,09hVIj6vWgoCDtT03h8ZCa,A Tribe Called Quest,66.0,2029407.0,"conscious hip hop, east coast hip hop, golden ...",2024-01-15,False
...,...,...,...,...,...,...,...
2093,6G9bygHlCyPgNGxK2l3YdE,NaN,NaN,NaN,NaN,NaN,True
2094,3sklFG9fuDAq3vbIZlkNH6,NaN,NaN,NaN,NaN,NaN,True
2095,2Cm6C9PNHioyjRKBfO7n9N,NaN,NaN,NaN,NaN,NaN,True
2096,0NIPkIjTV8mB795yEIiPYL,NaN,NaN,NaN,NaN,NaN,True


## Update all artists' info based on Spotify's records and add it to historical records

In [6]:
# Get artist info based on list of IDs
artistInfo = fetchArtistsInfo(credentials, artistsKnown['artistID'].to_list())
# Set the date of the data capture
artistInfo['date'] = date.today()
# Record whether artist was liked or not
artistInfo = artistInfo.merge(artistsKnown['artistID'], how='left', on='artistID')

# Read in historical
# Read in known artists from csv
if os.path.exists(artistsHistoricalPath):
    artistsHistorical = pd.read_csv(artistsHistoricalPath)
else:
    artistsHistorical = pd.DataFrame(columns=['date', 'following', 'artistName', 'artistID', 'artistPopularity','artistFollowers', 'artistGenre'])

# Merge in new data
artistsHistorical = pd.concat([artistsHistorical, artistInfo], axis=0, ignore_index=True)

artistsHistorical.head()

,date,following,artistName,artistID,artistPopularity,artistFollowers,artistGenre
0,2023-06-11,False,21 Savage,1URnnhqYAYcrqrcwql10ft,91,13740452,"atl hip hop, hip hop, rap, trap"
1,2023-06-11,False,3 Doors Down,2RTUTCvo6onsAnheUk3aL9,70,4079862,"alternative metal, nu metal, post-grunge, rock"
2,2023-06-11,False,311,41Q0HrwWBtuUkJc7C1Rp6K,64,949211,"alternative metal, funk metal, funk rock, nu m..."
3,2023-06-11,False,A Day To Remember,4NiJW4q9ichVqL1aUsgGAN,70,2219912,"alternative metal, metalcore, pop punk, screamo"
4,2023-06-11,False,A Tribe Called Quest,09hVIj6vWgoCDtT03h8ZCa,67,1897388,"conscious hip hop, east coast hip hop, golden ..."


## Load

In [8]:
# Write artistsKnown and artistsHistorical csvs
artistInfo.to_csv(artistsKnownPath, index=False)
artistsHistorical.to_csv(artistsHistoricalPath, index=False)